In [1]:
from __future__ import print_function
import numpy as np
import pandas as pd
import keras
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from keras import Sequential
from keras.layers import Input,Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
from keras.callbacks import ReduceLROnPlateau
from keras.models import Model
from keras.optimizers import Adam, RMSprop
from matplotlib import pyplot
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier
from hyperopt import Trials, STATUS_OK, tpe
from hyperas import optim
from hyperas.distributions import choice, uniform
np.random.seed(7)
%matplotlib inline  
#np.set_printoptions(threshold=np.nan)

C:\Users\Chris\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def data():
    # Split train & validation set
    FFF_Train_Decile9 = pd.read_pickle("FFF_Train_Decile9.pkl")
    FFF_Train_Targets_1D = pd.read_pickle("FFF_Train_Targets_1D.pkl")
    X_test = pd.read_pickle("FFF_Test_Decile9.pkl")
    y_test = pd.read_pickle("FFF_Test_Targets_1D.pkl").astype(int)
    
    X_train, X_val, y_train, y_val = train_test_split(FFF_Train_Decile9, FFF_Train_Targets_1D, test_size=0.20, random_state=7)
    X_train = X_train.astype('float32')
    X_val = X_val.astype('float32')
    return X_train, y_train, X_val, y_val, X_test, y_test


# Function to create model, required for KerasClassifier
def create_model(X_train, y_train, X_val, y_val, X_test, y_test):
    # create model
    model = Sequential()
    model.add(Masking(mask_value=-1, input_shape =(X_train.shape[1],3) ))
    model.add(LSTM(16, activation='tanh'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))

    # Compile model                                                   
    optimizer = Adam(lr={{choice([0.001,0.0005,0.0001])}})
    model.compile(loss='binary_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    

    
    # Fit model
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                                  patience=5, verbose = 1, min_lr=0.00001, mode = "min", min_delta=0.001)
    model.fit(X_train, y_train, epochs={{choice([50,100,150,200])}}, batch_size= 32, verbose=1,
                       validation_data = (X_val, y_val), callbacks = [reduce_lr])
    
    score, acc = model.evaluate(X_val, y_val, verbose=0)
    print('Test accuracy:', acc)
    return {'loss': -acc, 'status': STATUS_OK, 'model': model}

if __name__ == '__main__':
    best_run, best_model = optim.minimize(model=create_model,
                                          data=data,
                                          algo=tpe.suggest,
                                          max_evals=10,
                                          trials=Trials(),
                                          notebook_name='LSTM FFF Decile 9')
    X_train, y_train, X_val, y_val, X_test, y_test = data()
    print("Evalutation of best performing model:")
    print(best_model.evaluate(X_test, y_test))
    print("Best performing model chosen hyper-parameters:")
    print(best_run)
    
    y_pred = best_model.predict(X_test)
    y_classes = np.round(y_pred)
    print('Confusion Matrix for LSTM predictions:')
    print(confusion_matrix(y_pred=y_classes, y_true=y_test))
    print('Classification Report for LSTM predictions:')
    print('-------------------------------------------')
    print(classification_report(y_pred=y_classes, y_true=y_test))
    print("Accuracy score: " + str(accuracy_score(y_pred=y_classes, y_true=y_test)))

>>> Imports:
#coding=utf-8

from __future__ import print_function

try:
    import numpy as np
except:
    pass

try:
    import pandas as pd
except:
    pass

try:
    import keras
except:
    pass

try:
    from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
except:
    pass

try:
    from sklearn.model_selection import train_test_split
except:
    pass

try:
    from sklearn.utils import class_weight
except:
    pass

try:
    from keras import Sequential
except:
    pass

try:
    from keras.layers import Input, Embedding, LSTM, Dense, Masking, Dropout, BatchNormalization
except:
    pass

try:
    from keras.callbacks import ReduceLROnPlateau
except:
    pass

try:
    from keras.models import Model
except:
    pass

try:
    from keras.optimizers import Adam, RMSprop
except:
    pass

try:
    from matplotlib import pyplot
except:
    pass

try:
    from sklearn.model_selection import GridSearchCV
except:
    pass

try:
    from keras.wrappers.scik

3055/3055 [==============================] - 7s 2ms/step - loss: 0.2778 - acc: 0.8900 - val_loss: 0.2492 - val_acc: 0.8901
Epoch 39/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2727 - acc: 0.8953 - val_loss: 0.2478 - val_acc: 0.8940
Epoch 40/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2743 - acc: 0.8946 - val_loss: 0.2496 - val_acc: 0.8901
Epoch 41/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2713 - acc: 0.8979 - val_loss: 0.2484 - val_acc: 0.8992

Epoch 00041: ReduceLROnPlateau reducing learning rate to 1e-05.
Epoch 42/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2743 - acc: 0.8949 - val_loss: 0.2480 - val_acc: 0.8979
Epoch 43/100
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2726 - acc: 0.8926 - val_loss: 0.2478 - val_acc: 0.8966
Epoch 44/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2696 - acc: 0.8962 - val_loss: 0.2471 - val_acc: 0.8

Epoch 98/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2641 - acc: 0.8959 - val_loss: 0.2444 - val_acc: 0.8953
Epoch 99/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2660 - acc: 0.8975 - val_loss: 0.2448 - val_acc: 0.8953
Epoch 100/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2675 - acc: 0.8962 - val_loss: 0.2455 - val_acc: 0.8966
Test accuracy: 0.8965968577025448
Train on 3055 samples, validate on 764 samples
Epoch 1/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.5733 - acc: 0.6720 - val_loss: 0.4275 - val_acc: 0.8835
Epoch 2/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4060 - acc: 0.8736 - val_loss: 0.3384 - val_acc: 0.8861
Epoch 3/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3524 - acc: 0.8795 - val_loss: 0.2973 - val_acc: 0.8848
Epoch 4/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3126 - acc: 0.8828 - val_l

3055/3055 [==============================] - 7s 2ms/step - loss: 0.2193 - acc: 0.9110 - val_loss: 0.2121 - val_acc: 0.9097
Epoch 57/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2202 - acc: 0.9116 - val_loss: 0.2119 - val_acc: 0.9097
Epoch 58/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2197 - acc: 0.9129 - val_loss: 0.2116 - val_acc: 0.9084
Epoch 59/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2198 - acc: 0.9146 - val_loss: 0.2115 - val_acc: 0.9084
Epoch 60/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2198 - acc: 0.9142 - val_loss: 0.2117 - val_acc: 0.9084
Epoch 61/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2194 - acc: 0.9119 - val_loss: 0.2116 - val_acc: 0.9097
Epoch 62/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2219 - acc: 0.9100 - val_loss: 0.2118 - val_acc: 0.9084
Epoch 63/150
3055/3055 [==============================] - 7s 

3055/3055 [==============================] - 7s 2ms/step - loss: 0.2201 - acc: 0.9146 - val_loss: 0.2113 - val_acc: 0.9110
Epoch 117/150
3055/3055 [==============================] - 8s 2ms/step - loss: 0.2176 - acc: 0.9149 - val_loss: 0.2115 - val_acc: 0.9123
Epoch 118/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2171 - acc: 0.9133 - val_loss: 0.2118 - val_acc: 0.9110
Epoch 119/150
3055/3055 [==============================] - 8s 3ms/step - loss: 0.2200 - acc: 0.9136 - val_loss: 0.2116 - val_acc: 0.9097
Epoch 120/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2196 - acc: 0.9119 - val_loss: 0.2114 - val_acc: 0.9097
Epoch 121/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2196 - acc: 0.9106 - val_loss: 0.2119 - val_acc: 0.9097
Epoch 122/150
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2196 - acc: 0.9136 - val_loss: 0.2118 - val_acc: 0.9097
Epoch 123/150
3055/3055 [==============================

...

3055/3055 [==============================] - 7s 2ms/step - loss: 0.2729 - acc: 0.8998 - val_loss: 0.2587 - val_acc: 0.9005
Epoch 65/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2734 - acc: 0.8995 - val_loss: 0.2595 - val_acc: 0.8966
Epoch 66/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2763 - acc: 0.8975 - val_loss: 0.2596 - val_acc: 0.8966
Epoch 67/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2706 - acc: 0.8992 - val_loss: 0.2600 - val_acc: 0.8953
Epoch 68/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2726 - acc: 0.8998 - val_loss: 0.2595 - val_acc: 0.8953
Epoch 69/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2709 - acc: 0.9002 - val_loss: 0.2593 - val_acc: 0.8966
Epoch 70/100
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2764 - acc: 0.9005 - val_loss: 0.2602 - val_acc: 0.8953
Epoch 71/100
3055/3055 [==============================] - 7s 

Epoch 24/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2588 - acc: 0.8992 - val_loss: 0.2500 - val_acc: 0.8874
Epoch 25/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2507 - acc: 0.8995 - val_loss: 0.2368 - val_acc: 0.8940

Epoch 00025: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.
Epoch 26/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2419 - acc: 0.8989 - val_loss: 0.2262 - val_acc: 0.8992
Epoch 27/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2385 - acc: 0.9057 - val_loss: 0.2242 - val_acc: 0.8979
Epoch 28/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2337 - acc: 0.9064 - val_loss: 0.2233 - val_acc: 0.8992
Epoch 29/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2313 - acc: 0.9044 - val_loss: 0.2249 - val_acc: 0.9031
Epoch 30/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2372 - acc: 0.9070 - v

3055/3055 [==============================] - 7s 2ms/step - loss: 0.2292 - acc: 0.9067 - val_loss: 0.2215 - val_acc: 0.9018
Epoch 83/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2343 - acc: 0.9005 - val_loss: 0.2215 - val_acc: 0.9018
Epoch 84/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2288 - acc: 0.9051 - val_loss: 0.2213 - val_acc: 0.9018
Epoch 85/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2299 - acc: 0.9093 - val_loss: 0.2214 - val_acc: 0.9018
Epoch 86/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2278 - acc: 0.9087 - val_loss: 0.2214 - val_acc: 0.9018
Epoch 87/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2289 - acc: 0.9061 - val_loss: 0.2214 - val_acc: 0.9018
Epoch 88/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2298 - acc: 0.9097 - val_loss: 0.2214 - val_acc: 0.9018
Epoch 89/200
3055/3055 [==============================] - 7s 

3055/3055 [==============================] - 6s 2ms/step - loss: 0.2262 - acc: 0.9087 - val_loss: 0.2213 - val_acc: 0.9005
Epoch 143/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2318 - acc: 0.9080 - val_loss: 0.2214 - val_acc: 0.8992
Epoch 144/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2320 - acc: 0.9064 - val_loss: 0.2214 - val_acc: 0.8992
Epoch 145/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2355 - acc: 0.9080 - val_loss: 0.2214 - val_acc: 0.9005
Epoch 146/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2302 - acc: 0.9090 - val_loss: 0.2214 - val_acc: 0.9005
Epoch 147/200
3055/3055 [==============================] - 6s 2ms/step - loss: 0.2282 - acc: 0.9087 - val_loss: 0.2214 - val_acc: 0.8992
Epoch 148/200
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2315 - acc: 0.9083 - val_loss: 0.2214 - val_acc: 0.8992
Epoch 149/200
3055/3055 [==============================

3055/3055 [==============================] - 8s 3ms/step - loss: 0.6208 - acc: 0.6242 - val_loss: 0.5308 - val_acc: 0.6846
Epoch 2/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4819 - acc: 0.7718 - val_loss: 0.4033 - val_acc: 0.8613
Epoch 3/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.4111 - acc: 0.8589 - val_loss: 0.3528 - val_acc: 0.8717
Epoch 4/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3534 - acc: 0.8704 - val_loss: 0.3011 - val_acc: 0.8822
Epoch 5/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3212 - acc: 0.8782 - val_loss: 0.2864 - val_acc: 0.8861
Epoch 6/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.3112 - acc: 0.8848 - val_loss: 0.2779 - val_acc: 0.8887
Epoch 7/50
3055/3055 [==============================] - 7s 2ms/step - loss: 0.2935 - acc: 0.8903 - val_loss: 0.2654 - val_acc: 0.8979
Epoch 8/50
3055/3055 [==============================] - 7s 2ms/step - los

In [ ]:
0.93484

In [ ]:
#0.91689